In [2]:
# Install required libraries
!pip install transformers datasets

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

In [4]:
dataset = load_dataset("imdb")
train_data = dataset["train"].shuffle(seed=42).select(range(200))  # Use only 200 samples
test_data = dataset["test"].shuffle(seed=42).select(range(100))   # Use only 100 samples

In [5]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def preprocess(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=64)  # Shorter max_length for speed

train_data = train_data.map(preprocess, batched=True)
test_data = test_data.map(preprocess, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    output_dir="./fast_results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1,
    logging_steps=5,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.664800,0.644832


TrainOutput(global_step=25, training_loss=0.6699230098724365, metrics={'train_runtime': 157.0801, 'train_samples_per_second': 1.273, 'train_steps_per_second': 0.159, 'total_flos': 3311684966400.0, 'train_loss': 0.6699230098724365, 'epoch': 1.0})

In [10]:
trainer.save_model("./fast_fine_tuned_model")

In [11]:
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.6448318362236023, 'eval_runtime': 10.8118, 'eval_samples_per_second': 9.249, 'eval_steps_per_second': 1.202, 'epoch': 1.0}
